<a href="https://colab.research.google.com/github/T-Sunm/Learn-Data-Cleaning-in-kaggle/blob/main/Exercise_Handling_Missing_Values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'building-permit-applications-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F16432%2F21688%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240825%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240825T165128Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dc0c874c48f7bea623549740da1ae3568fd75dfb3106414f1b4f2a26deb3bc3b81920d4d3cee5f4cc9bd48409d82ebc5b892ae15319640828651b967eff31613a565c4434af16f5fd87ba762ed1d822abf1cb29d091fadcda8f2b93754fd2b1afc39d7d9e1b10d2e7723844cb1f579d757f96db573995f76e0e3ab75a81ea151adec34e5c6fbc5d7a49f600b1f6454a707b7fd676fc8064f3055d00cdd95144a7c67e5d6ea2c555e84e76ad1d0de0b2be01c1ef4d5940c0d50b9d94016f014634f10334e25c6d5e3e35f1ca75d4b57068385e81ad03d5656c9cb4e79db7f0f45519eff47cdbb470fcc75f60b3fbbeb8b0deb7e00fced08612185166bd9374c03c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Data Cleaning](https://www.kaggle.com/learn/data-cleaning) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/handling-missing-values).**

---


In this exercise, you'll apply what you learned in the **Handling missing values** tutorial.

# Setup

The questions below will give you feedback on your work. Run the following cell to set up the feedback system.

In [ ]:
from learntools.core import binder
binder.bind(globals())
from learntools.data_cleaning.ex1 import *
print("Setup Complete")

# 1) Take a first look at the data

Run the next code cell to load in the libraries and dataset you'll use to complete the exercise.

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np

# read in all our data
sf_permits = pd.read_csv("../input/building-permit-applications-data/Building_Permits.csv")

# set seed for reproducibility
np.random.seed(0)

Use the code cell below to print the first five rows of the `sf_permits` DataFrame.

In [ ]:
sf_permits.head()


Does the dataset have any missing values?  Once you have an answer, run the code cell below to get credit for your work.

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q1.check()

In [ ]:
# Line below will give you a hint
#q1.hint()

# 2) How many missing data points do we have?

What percentage of the values in the dataset are missing?  Your answer should be a number between 0 and 100.  (If 1/4 of the values in the dataset are missing, the answer is 25.)

In [ ]:
sf_permits.isnull().sum().sum()

In [ ]:
# TODO: Your code here!
percent_missing = sf_permits.isnull().sum().sum() / np.product(sf_permits.shape) * 100

# Check your answer
q2.check()

In [ ]:
# Lines below will give you a hint or solution code
#q2.hint()
#q2.solution()

# 3) Figure out why the data is missing

Look at the columns **"Street Number Suffix"** and **"Zipcode"** from the [San Francisco Building Permits dataset](https://www.kaggle.com/aparnashastry/building-permit-applications-data). Both of these contain missing values.
- Which, if either, are missing because they don't exist?
- Which, if either, are missing because they weren't recorded?  

Once you have an answer, run the code cell below.

- Street Number Suffix
- Zipcode

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q3.check()

In [ ]:
# Line below will give you a hint
#q3.hint()

# 4) Drop missing values: rows

If you removed all of the rows of `sf_permits` with missing values, how many rows are left?

**Note**: Do not change the value of `sf_permits` when checking this.  

In [ ]:
sf_permits.dropna()


Once you have an answer, run the code cell below.

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q4.check()

In [ ]:
# Line below will give you a hint
#q4.hint()

# 5) Drop missing values: columns

Now try removing all the columns with empty values.  
- Create a new DataFrame called `sf_permits_with_na_dropped` that has all of the columns with empty values removed.  
- How many columns were removed from the original `sf_permits` DataFrame? Use this number to set the value of the `dropped_columns` variable below.

In [ ]:
permit_c_col = sf_permits.isnull().sum()
name_permit_null = permit_c_col[permit_c_col  0]
name_permit_null.index.to_list()

In [ ]:
permit_c_col = sf_permits.isnull().sum()
name_permit_null = permit_c_col[permit_c_col == 0].index
# TODO: Your code here
sf_permits_with_na_dropped = sf_permits[name_permit_null]

dropped_columns = sf_permits.columns.shape[0] - sf_permits.dropna(axis = 1).columns.shape[0]

# Check your answer
q5.check()

In [ ]:
# Lines below will give you a hint or solution code
#q5.hint()
#q5.solution()

# 6) Fill in missing values automatically

Try replacing all the NaN's in the `sf_permits` data with the one that comes directly after it and then replacing any remaining NaN's with 0.  Set the result to a new DataFrame `sf_permits_with_na_imputed`.

In [ ]:
# TODO: Your code here
sf_permits_with_na_imputed = sf_permits.fillna(method = 'bfill', axis = 0).fillna(0)

# Check your answer
q6.check()

In [ ]:
# Lines below will give you a hint or solution code
#q6.hint()
#q6.solution()

# More practice

If you're looking for more practice handling missing values:

* Check out [this noteboook](https://www.kaggle.com/alexisbcook/missing-values) on handling missing values using scikit-learn's imputer.
* Look back at the "Zipcode" column in the `sf_permits` dataset, which has some missing values. How would you go about figuring out what the actual zipcode of each address should be? (You might try using another dataset. You can search for datasets about San Fransisco on the [Datasets listing](https://www.kaggle.com/datasets).)

# Keep going

In the next lesson, learn how to [**apply scaling and normalization**](https://www.kaggle.com/alexisbcook/scaling-and-normalization) to transform your data.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/data-cleaning/discussion) to chat with other learners.*